## Example NRS import

This notebook will import recordings in .nrs format from the cloud

In [1]:
print('hi')

hi


In [5]:
# Define some functions

import lbdb
import kachery as ka
import labbox_ephys as le
import hither
import numpy as np

def geom_from_recording(recording):
    channel_ids = recording.get_channel_ids()
    location0 = recording.get_channel_property(channel_ids[0], 'location')
    nd = len(location0)
    M = len(channel_ids)
    geom = np.zeros((M, nd))
    for ii in range(len(channel_ids)):
        location_ii = recording.get_channel_property(channel_ids[ii], 'location')
        geom[ii, :] = list(location_ii)
    return geom

def import_recording(*, recording_id, recording):
    print(f'Importing {recording_id}')
    print('--- obj', recording.object())
    doc = dict(
        recording_id=recording_id,
        recording=recording.object(),
        sampling_frequency=recording.get_sampling_frequency(),
        channel_ids=recording.get_channel_ids(),
        channel_groups=recording.get_channel_groups(),
        geom=geom_from_recording(recording).tolist(),
        num_frames=recording.get_num_frames()
    )
    le.dbcollection('recordings').replace_one(dict(recording_id=recording_id), doc, upsert=True)

In [6]:
# Import recordings from cloud

with ka.config(fr='default_readonly'):
    #
    import_recording(
        recording_id='nrs:128chan_sampleCA1',
        recording=le.LabboxEphysRecordingExtractor('sha1dir://80b58d66a95546365f2803993794f6cb7d7d31f8.128chan_sampleCA1')
    )

Importing nrs:128chan_sampleCA1
--- obj sha1dir://80b58d66a95546365f2803993794f6cb7d7d31f8.128chan_sampleCA1


In [4]:
# List all of the recordings in the database

recordings = le.dbcollection('recordings').find(dict())
for rec in recordings:
    print(rec['recording_id'])

sf:synth_magland_noise10_K10_C4/001_synth
sf:paired_kampff/2014_11_25_Pair_3_0
sf:paired_boyden32c/1103_1_1
nrs:128chan_sampleCA1
